<a href="https://colab.research.google.com/github/migostro/laboratorio-de-visao/blob/main/3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução

Esta é a última etapa do trabalho e está relacionada À classificação automática das imagens. Este é um trabalho relacionado ao Exercício Programa da disciplina MAC417 - Visão e Processamento de Imagens. As imagens coletadas e geradas por outras etapas, assim como os códigos referentes a cada etapa, encontram-se no nosso repositório do [Google Drive](https://https://colab.research.google.com/drive/1gpEmM9aYBCf2G8a5cdOZweZeCxlnBZIe#scrollTo=pdNd6kmXs27k&line=4&uniqifier=1). Os integrantes do grupo são:

* Mateus Santos Freire NUSP: 11796889
* Arthur Sakayan NUSP: 10297647
* Aldomar Pietro Santana Silva NUSP: 10770162
* Miguel Pereira Ostrowski NUSP: 10723610
* Eduarda Ramos Bezerra de Alencar NUSP:10372540
* Rafael Badain NUSP: 10277102

#Imports

In [ ]:
!pip install scikit-image==0.19.2

In [ ]:
import numpy as np
from google.colab import files, drive
import os
from skimage import io, color
from skimage import segmentation, measure

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Helper Functions

A função auxiliar desenvolvida pelo grupo que será utilizada nesta etapa é a list_filepaths(), que lista todos os caminhos de arquivos em uma pasta raíz dada.

In [ ]:
# list_filepaths(): list filepath for every filepath on a root folder
# pre-condition: (root path, empty list)
# post-condition: list with every filepath on root path
def list_filepaths(path, file_extension, filepaths = []):
    if os.path.isdir(path):
        for filename in os.listdir(path):
            filepath = os.path.join(path, filename)
            if os.path.isfile(filepath) and file_extension in filename: filepaths.append(filepath) # Adiciona apenas caminhos que são de arquivos (que no nosso caso são imagens)
            else: list_filepaths(filepath, file_extension, filepaths)

    return filepaths

# Pipeline

Aqui iniciamos o processo de gerenciamento e processamento das imagens. As células a seguir estão responsáveis pela definição de datasets que serão utilizados, listagem das imagens e suas características, como número de classes e de imagens.

In [ ]:
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new'
resizedData_path = root + '/originalDataSet_resized'
truth_path = root + "/groundTruth_resized"
segmented_path = root + "/segmented_resized"

resizedData_filepaths = list_filepaths(resizedData_path, 'png', [])
truthData_filepaths = list_filepaths(truth_path, 'png', [])
segmentedData_filepaths = list_filepaths(segmented_path, 'png', [])
print(f'# of images @ {resizedData_path}: {len(resizedData_filepaths)}')
print(f'# of images @ {truth_path}: {len(truthData_filepaths)}')
print(f'# of images @ {segmented_path}: {len(segmentedData_filepaths)}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized: 1080
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/groundTruth_resized: 160
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/segmented_resized: 1080


In [ ]:
# 0 FILE READING

drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new'
#root = '/content/drive/MyDrive/ime/mac0417/ep'
originalPath = root + "/originalDataSet_resized"
segmented_path = root + "/segmented_resized"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
originalPaths = list_filepaths(originalPath, 'png', [])
segmentedPaths = []
for img_path in originalPaths:
    segmentedPaths.append(img_path.replace(originalPath, segmented_path))

print(f'# of images @ {originalPath}: {len(originalPaths)}')
print(f'# of images @ {segmented_path}: {len(segmentedPaths)}')
print(originalPaths)
print(segmentedPaths)

# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized: 1080
# of images @ /content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/segmented_resized: 1080
['/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized/noite/interior/branco/cartas/cartas_002/IMG_1608.png', '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized/noite/interior/branco/cartas/cartas_002/IMG_1607.png', '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized/noite/interior/branco/cartas/cartas_002/IMG_1606.png', '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized/noite/interior/branco/cartas/cartas_003/IMG_1614.png', '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized/noite/interior/branco/cartas/cartas_003/IMG_1613.png', '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new/originalDataSet_resized/noite/interior/branco/cartas/cartas_003/IMG_1612.png', '/content/drive/MyDri

In [ ]:
classes = ['borrachas', 'cartas', 'celulares', 'conchas', 'copos', 'dados', 'estatuas', 'lapis', 'tubos', 'vasos']
num_samples = len(originalPaths)
num_features = 9
num_classes = len(classes)
X = np.zeros([num_samples, num_features])
Y = np.zeros([num_samples, num_classes])

# https://scikit-learn.org/stable/modules/feature_extraction.html
for i in range(num_samples):
    for j in range(num_classes):
        if originalPaths[i].split('/')[-3] == classes[j]:
            Y[i, j] = 1

## Creating the feature vector

O feature vector será utilizado para treinar nosso modelo classificador. Assim, para cada imagem retiramos propriedadoes como área, solidez, perímetro, orientação, entre outros.

In [ ]:
# create feature vector
for i in range(num_samples):
    print(f'[{i}]')
    img_original = io.imread(originalPaths[i])
    img_segmented = io.imread(segmentedPaths[i])
    cut = np.multiply(img_original, img_segmented)

    label_img = measure.label(cut)
    regions = measure.regionprops(label_img)

    if(len(regions) > 1): # isolate_largest_area
        largest_region = max(regions, key=lambda region: region.area)

    if len(regions) == 1:
        largest_region = regions[0]
        
    X[i, 0] = largest_region.area
    X[i, 1] = largest_region.solidity
    X[i, 2] = largest_region.eccentricity
    X[i, 3] = largest_region.feret_diameter_max
    X[i, 4] = largest_region.perimeter
    X[i, 5] = largest_region.orientation
    X[i, 6] = largest_region.extent
    X[i, 7] = largest_region.axis_major_length
    X[i, 8] = largest_region.axis_minor_length

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[115]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[139]
[140]
[141]
[142]
[143]
[144]
[145]
[146]
[147]
[148]
[149]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]
[158]
[159]
[160]
[161]
[162]
[163]
[164]
[165]
[166]
[167]
[168]
[169]
[170]
[171]
[172]
[173]
[174]
[175]
[176]
[177]
[178]
[179]
[180]
[181]
[182]
[183]
[184]


In [ ]:
print(X.shape, Y.shape)

(1080, 9) (1080, 10)


##Encontrando uma boa combinação de alpha, número de camadas e epislom
Aqui nós testamos todas as combinações entre alguns valores arbitrários para cada hiperparâmetro, escolhendo a melhor combinação que obteve o maior score. O modelo escolhido foi o Multilayer Perceptron, um modelo de rede neural que irá ponderar as features dadas na entrada e, como saída, irá retornar a classe que melhor se adequa para as características fornecidas.

In [ ]:
alphas = np.arange(0.1, 1.1, 0.1)
epsilons = np.array([1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8])
layers = np.arange(10, 101, 10)

print(alphas)
print(epsilons)
print(layers)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[1.e-03 1.e-04 1.e-05 1.e-06 1.e-07 1.e-08]
[ 10  20  30  40  50  60  70  80  90 100]


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

alphas = np.arange(0.4, 1.1, 0.1)
epsilons = np.array([1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8])
layers = np.arange(10, 101, 10)

cases = []
for a in range(len(alphas)):
    for e in range(len(epsilons)):
        for l in range(len(layers)):
            
            # create the classifier
            classifier = MLPClassifier(solver='adam', alpha=alphas[a], epsilon=epsilons[e], hidden_layer_sizes=(layers[l]), max_iter=10000, n_iter_no_change=50, random_state=1)

            # train the model
            classifier.fit(X_train, Y_train)

            # calculate the avg score
            score = classifier.score(X_test, Y_test)

            vetor = []

            vetor.append(alphas[a])
            vetor.append(epsilons[e])
            vetor.append(layers[l])
            vetor.append(score)

            cases.append(vetor)
    print(a)
            


0
1
2
3
4
5
6


In [ ]:
print(cases)

[[0.4, 0.001, 10, 0.06666666666666667], [0.4, 0.001, 20, 0.07407407407407407], [0.4, 0.001, 30, 0.08148148148148149], [0.4, 0.001, 40, 0.011111111111111112], [0.4, 0.001, 50, 0.0962962962962963], [0.4, 0.001, 60, 0.1037037037037037], [0.4, 0.001, 70, 0.06296296296296296], [0.4, 0.001, 80, 0.05925925925925926], [0.4, 0.001, 90, 0.04814814814814815], [0.4, 0.001, 100, 0.12962962962962962], [0.4, 0.0001, 10, 0.08518518518518518], [0.4, 0.0001, 20, 0.07777777777777778], [0.4, 0.0001, 30, 0.11851851851851852], [0.4, 0.0001, 40, 0.08888888888888889], [0.4, 0.0001, 50, 0.1037037037037037], [0.4, 0.0001, 60, 0.07777777777777778], [0.4, 0.0001, 70, 0.08148148148148149], [0.4, 0.0001, 80, 0.07037037037037037], [0.4, 0.0001, 90, 0.11851851851851852], [0.4, 0.0001, 100, 0.0962962962962963], [0.4, 1e-05, 10, 0.04814814814814815], [0.4, 1e-05, 20, 0.08518518518518518], [0.4, 1e-05, 30, 0.09259259259259259], [0.4, 1e-05, 40, 0.06296296296296296], [0.4, 1e-05, 50, 0.05555555555555555], [0.4, 1e-05, 60

In [ ]:
maior = 0
for i in range(len(cases)):
    if cases[i][3] > cases[maior][3]:
        maior = i
print(cases[maior][3])
print(cases[maior][:])

0.2037037037037037
[0.4, 1e-08, 80, 0.2037037037037037]


## Results
No final, nosso score geral foi de aproximadamente 20% de acerto, não sendo um resultado bom. Acreditamos que não tenha sido bom, já que, ao se plotar alguns dados juntos, não foi possível ver uma distinção clara das classes, tendo agrupamentos de classes muito próximos um dos outros, dificultando muito a classificação. E em segundo, acreditamos que o fator anterior foi amplificado em imagens que não tiveram uma segmentação muito boa, dado a dificuladade do problema.

In [ ]:
a = cases[maior][0]
e = cases[maior][1]
l = cases[maior][2]

best_clf = MLPClassifier(alpha=a, epsilon=e, hidden_layer_sizes=(l), max_iter=10000, n_iter_no_change=50, random_state=1)
best_clf.fit(X_train, Y_train)

print("Score using the train set", best_clf.score(X_train, Y_train))
print("Score using the test", best_clf.score(X_test, Y_test))
print("General score", best_clf.score(X, Y))

Score using the train set 0.22592592592592592
Score using the test 0.2037037037037037
General score 0.22037037037037038
